In [15]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple

def gaussian_upper(A: np.ndarray, b: np.ndarray, tol: float = 1e-12) -> np.ndarray:
    A = A.astype(float, copy=True)
    b = b.astype(float).reshape(-1, 1)
    #Isplestine matrica
    Ab = np.hstack([A, b])
    m, n_plus_1 = Ab.shape
    n = n_plus_1 - 1
    i = 0

    #Einame per stulpelius ir darome Gauso matrica
    for j in range(n):
        #Pasirenkame stulpelyje didžiausią pagal modulį elementą kaip pagrindą
        pivot = i + np.argmax(np.abs(Ab[i:, j]))
        
        #Jei pivotas beveik nulis – stulpelyje nėra naudingos informacijos
        if np.abs(Ab[pivot, j]) < tol:
            continue

        #Jei reikia – sukeičiam eilutes, kad pivotas atsidurtų viršuje
        if pivot != i:
            Ab[[i, pivot], :] = Ab[[pivot, i], :]

        #Nuliname žemiau pivoto esančius elementus
        for r in range(i+1, m):
            if np.abs(Ab[r, j]) > tol:
                factor = Ab[r, j] / Ab[i, j]
                Ab[r, j:] -= factor * Ab[i, j:]

        #Pereiname prie kitos eilutes
        i += 1
        
        if i == m:
            break

    return Ab
    
def lu_solve(P: np.ndarray, L: np.ndarray, U: np.ndarray, b: np.ndarray) -> np.ndarray:
    #Pritaikome permutaciją dešiniajam nariui: Pb = P * b
    Pb = P @ b
    y = np.zeros_like(b, dtype=float)
    x = np.zeros_like(b, dtype=float)

    #Išsprendžiame L y = P b (priekinei eigai nereikia dalybos iš įstrižainės, nes L turi 1)
    for i in range(L.shape[0]):
        y[i] = Pb[i] - L[i, :i] @ y[:i]

    #Išsprendžiame U x = y (atgalinė eiga)
    for i in range(U.shape[0]-1, -1, -1):
        if abs(U[i, i]) < 1e-12:
            raise ValueError("Singuliarumo požymis.")
            
        x[i] = (y[i] - U[i, i+1:] @ x[i+1:]) / U[i, i]
    
    return x

def lu_factor_pp(A: np.ndarray, tol: float = 1e-12) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    #LU išskaidymas su daliniu perrinkimu: P*A = L*U
    A = A.astype(float).copy()
    n = A.shape[0]
    P = np.eye(n)
    L = np.zeros((n, n))
    U = A.copy()
    
    for k in range(n):
        #Randame didžiausią elementą stulpelyje k nuo eilutės k žemyn
        pivot = np.argmax(np.abs(U[k:, k])) + k
        
        #Jei pivotas ~0 – matrica singuliari ar beveik singuliari
        if abs(U[pivot, k]) < tol:
            raise ValueError("Matrica artima singuliariai.")

        #Perkeliame pivot eilutę į viršų ir atitinkamai atnaujiname P bei L
        if pivot != k:
            U[[k, pivot], :] = U[[pivot, k], :]
            P[[k, pivot], :] = P[[pivot, k], :]
            
            if k > 0:
                L[[k, pivot], :k] = L[[pivot, k], :k]

        #Skaičiuojame daugiklius (L) ir nuliname žemiau U įstrižainės
        for i in range(k+1, n):
            L[i, k] = U[i, k] / U[k, k]
            U[i, k:] = U[i, k:] - L[i, k] * U[k, k:]

    #Įrašome vienetus ant L įstrižainės
    np.fill_diagonal(L, 1.0)
    
    return P, L, U

def solve_system(A: np.ndarray, b: np.ndarray, name: str):
    print(f"\n=== Sistema: {name} ===")
    
    A = A.astype(float)
    b = b.astype(float).reshape(-1)
    n = A.shape[1]

    print("Lygčių sistema:")
    for i in range(A.shape[0]):
        lygtis = []
        
        for j in range(A.shape[1]):
            coef = A[i, j]
            
            if j == 0:
                term = f"{coef:.2f}x{j+1}"
            else:
                term = f" {'+' if coef >= 0 else '-'} {abs(coef):.2f}x{j+1}"
            
            lygtis.append(term)
        
        print("".join(lygtis) + f" = {b[i]:.2f}")

    print("\nMatrica A:")
    print(A)

    print("\nVektorius b:")
    print(b)

    try:
        P, L, U = lu_factor_pp(A)
        x = lu_solve(P, L, U, b)

        print("\nLU Rezultatai:")
        print(x)
    except ValueError:
        print("\nLU nepavyko (matrica singuliari arba beveik singuliari). Pereiname prie Gauso metodo.")
        
        Ab = gaussian_upper(A, b)
        m, n1 = Ab.shape
        n = n1 - 1
        tol = 1e-12

        #Tikriname, ar nėra prieštaravimo: 0 ... 0 | c (c ≠ 0)
        contradiction = any(np.all(np.abs(Ab[i, :n]) <= tol) and (abs(Ab[i, -1]) > tol) for i in range(m))
        
        if contradiction:
            print("Išvada: sprendinių nėra.")
            x = None
        else:
            rA = sum(np.any(np.abs(Ab[i, :n]) > tol) for i in range(m))
        
            if rA == n:
                print("Išvada: vienintelis sprendinys.")
                x = np.zeros(n, dtype=float)
                
                for i in range(m-1, -1, -1):
                    row = Ab[i, :n]

                    #Praleidžiame visiškai nulinę eilutę
                    if not np.any(np.abs(row) > tol):
                        continue

                    #Pirmasis nenulinis elementas eilutėje – pagrindinis stulpelis
                    j = int(np.argmax(np.abs(row) > tol))
                    s = 0.0

                    #Sukaupiame žinomą dalį iš dešinės
                    if j+1 < n:
                        s = float(row[j+1:] @ x[j+1:])
                    
                    x[j] = (Ab[i, -1] - s) / row[j]
                
                print("Gauso Rezultatai x:")
                print(x)
            else:
                print("Išvada: be galo daug sprendinių.")
                pivot_cols = []

                #Surenkame visų eilučių pagrindinių stulpelių indeksus
                for i in range(m):
                    row = Ab[i, :n]
                    
                    if np.any(np.abs(row) > tol):
                        pivot_cols.append(int(np.argmax(np.abs(row) > tol)))
                        
                pivot_cols = sorted(set(pivot_cols))
                free_cols = [j for j in range(n) if j not in pivot_cols]
        
                x = np.zeros(n, dtype=float)

                #Parenkame vieną laisvąjį kintamąjį = 1 (paprastas konkretizavimas)
                if len(free_cols) > 0:
                    last_free = free_cols[-1]
                    x[last_free] = 1.0

                #Atgalinė eiga, skaičiuojant pagrindinius kintamuosius pagal pasirinktus laisvuosius
                for i in range(m-1, -1, -1):
                    row = Ab[i, :n]
                    
                    if not np.any(np.abs(row) > tol):
                        continue
                        
                    j = int(np.argmax(np.abs(row) > tol))
                    s = 0.0
                    
                    if j+1 < n:
                        s = float(row[j+1:] @ x[j+1:])
                        
                    if abs(row[j]) > tol:
                        x[j] = (Ab[i, -1] - s) / row[j]
        
                print("\nParinktas konkretus sprendinys:")
                print(x)

    if x is not None:
        #Patikriname liekaną r = A x - b
        r = A @ x - b

        print("\nAx = b")
        print(r)

        #Įvertiname sprendinio tikslumą pagal normą poelementiškai
        if np.any(np.abs(r) > 1e-9):
            print("Rasti rezultatai nebuvo pakankamai tikslūs (|Ax - b| > 1e-9)")
        else:
            print("Rasti rezultatai yra pakankamai tikslūs (|Ax - b| ≤ 1e-9)")

        try:
            x_np = np.linalg.solve(A, b)
            print("\nRezultatai naudojant standartines funkcijas:")
            print(x_np)
        except Exception:
            x_np, *_ = np.linalg.lstsq(A, b, rcond=1e-9)
            print("\nRezultatai naudojant standartines funkcijas:")
            print(x_np)
    else:
        r = None

if __name__ == "__main__":
    A1 = np.array([[3,10,1,5],[-2,6,12,14],[3,12,5,1],[-3,-9,5,0]], dtype=float)
    b1 = np.array([83,178,37,-26], dtype=float)
    A2 = np.array([[3,1,-1,5],[-3,4,-8,-1],[1,-3,7,6],[0,5,-9,4]], dtype=float)
    b2 = np.array([20,-36,41,-16], dtype=float)
    A3 = np.array([[2,5,1,2],[-2,0,3,5],[1,0,-1,1],[0,5,4,7]], dtype=float)
    b3 = np.array([-1,7,3,4], dtype=float)
    solve_system(A1, b1, "Sistema #1")
    solve_system(A2, b2, "Sistema #2")
    solve_system(A3, b3, "Sistema #3")



=== Sistema: Sistema #1 ===
Lygčių sistema:
3.00x1 + 10.00x2 + 1.00x3 + 5.00x4 = 83.00
-2.00x1 + 6.00x2 + 12.00x3 + 14.00x4 = 178.00
3.00x1 + 12.00x2 + 5.00x3 + 1.00x4 = 37.00
-3.00x1 - 9.00x2 + 5.00x3 + 0.00x4 = -26.00

Matrica A:
[[ 3. 10.  1.  5.]
 [-2.  6. 12. 14.]
 [ 3. 12.  5.  1.]
 [-3. -9.  5.  0.]]

Vektorius b:
[ 83. 178.  37. -26.]

LU Rezultatai:
[-2.  3. -1. 12.]

Ax = b
[0. 0. 0. 0.]
Rasti rezultatai yra pakankamai tikslūs (|Ax - b| ≤ 1e-9)

Rezultatai naudojant standartines funkcijas:
[-2.  3. -1. 12.]

=== Sistema: Sistema #2 ===
Lygčių sistema:
3.00x1 + 1.00x2 - 1.00x3 + 5.00x4 = 20.00
-3.00x1 + 4.00x2 - 8.00x3 - 1.00x4 = -36.00
1.00x1 - 3.00x2 + 7.00x3 + 6.00x4 = 41.00
0.00x1 + 5.00x2 - 9.00x3 + 4.00x4 = -16.00

Matrica A:
[[ 3.  1. -1.  5.]
 [-3.  4. -8. -1.]
 [ 1. -3.  7.  6.]
 [ 0.  5. -9.  4.]]

Vektorius b:
[ 20. -36.  41. -16.]

LU nepavyko (matrica singuliari arba beveik singuliari). Pereiname prie Gauso metodo.
Išvada: be galo daug sprendinių.

Parinktas konk